In [1]:
import numpy as np
import tensorflow as tf
import deepchem as dc

/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
"""
OFFSIDES dataset loader.
"""
from __future__ import division
from __future__ import unicode_literals

import os
import logging
import deepchem

logger = logging.getLogger(__name__)


def load_offsides(featurizer='ECFP', split='index', reload=True, K=4):
  logger.info("About to load ofssides dataset.")
  data_dir = deepchem.utils.get_data_dir()
  if reload:
    save_dir = os.path.join(data_dir, "offsides/" + featurizer + "/" + str(split))

  dataset_file = os.path.join("/home/mamonteiro/source-code/Project-LEI/sider+offsides/", "sider+offsides_combined.csv.gz")


  dataset = deepchem.utils.save.load_from_disk(dataset_file)
  logger.info("Columns of dataset: %s" % str(dataset.columns.values))
  logger.info("Number of examples in dataset: %s" % str(dataset.shape[0]))
  OFFSIDES_tasks = dataset.columns.values[1:].tolist()

  if reload:
    loaded, all_dataset, transformers = deepchem.utils.save.load_dataset_from_disk(
        save_dir)
    if loaded:
      return OFFSIDES_tasks, all_dataset, transformers

  # Featurize OFFSIDES dataset
  logger.info("About to featurize OFFSIDES dataset.")
  if featurizer == 'ECFP':
    featurizer = deepchem.feat.CircularFingerprint(size=1024)
  elif featurizer == 'GraphConv':
    featurizer = deepchem.feat.ConvMolFeaturizer()
  elif featurizer == 'Weave':
    featurizer = deepchem.feat.WeaveFeaturizer()
  elif featurizer == 'Raw':
    featurizer = deepchem.feat.RawFeaturizer()

  logger.info("OFFSIDES tasks: %s" % str(OFFSIDES_tasks))
  logger.info("%d tasks in total" % len(OFFSIDES_tasks))

  loader = deepchem.data.CSVLoader(
      tasks=OFFSIDES_tasks, smiles_field="smiles", featurizer=featurizer)
  dataset = loader.featurize(dataset_file)
  logger.info("%d datapoints in OFFSIDES dataset" % len(dataset))

  # Initialize transformers
  transformers = [
      deepchem.trans.BalancingTransformer(transform_w=True, dataset=dataset)
  ]
  logger.info("About to transform data")
  for transformer in transformers:
    dataset = transformer.transform(dataset)

  if split == None:
    return OFFSIDES_tasks, (dataset, None, None), transformers

  splitters = {
      'index': deepchem.splits.IndexSplitter(),
      'random': deepchem.splits.RandomSplitter(),
      'scaffold': deepchem.splits.ScaffoldSplitter(),
      'task': deepchem.splits.TaskSplitter()
  }
  splitter = splitters[split]
  if split == 'task':
    fold_datasets = splitter.k_fold_split(dataset, K)
    all_dataset = fold_datasets
  else:
    train, valid, test = splitter.train_valid_test_split(dataset)
    if reload:
      deepchem.utils.save.save_dataset_to_disk(save_dir, train, valid, test,
                                               transformers)
    all_dataset = (train, valid, test)
  return OFFSIDES_tasks, all_dataset, transformers


In [3]:
offsides_tasks, offsides_datasets, transformers = load_offsides(featurizer='GraphConv',reload=True)
train_dataset, valid_dataset, test_dataset = offsides_datasets

Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.


In [4]:
featurizer = dc.feat.CircularFingerprint(size = 1024)

In [5]:
loader = dc.data.CSVLoader(
      tasks=offsides_tasks, smiles_field="smiles",
      featurizer=featurizer)

In [6]:
dataset = loader.featurize('sider+offsides_combined.csv')

Loading raw samples now.
shard_size: 8192
About to start loading CSV from sider+offsides_combined.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 2.259 s
TIMING: dataset construction took 2.321 s
Loading dataset from disk.


In [7]:
train_dataset.y.shape

(704, 27)

In [8]:
valid_dataset.y.shape

(88, 27)

In [9]:
test_dataset.y.shape

(89, 27)

In [10]:
n_features = train_dataset.y.shape[0]
n_features

704

In [11]:
params_dict = {"activation": ["relu","sigmoid","tahn"],
               "optimizer": ["Adam","RMSprop"],
               "momentum": [.9],
               "penalty": [0.]
              }
n_features = train_dataset.y.shape[0]
def model_builder(model_params, model_dir):
    model = dc.models.MultitaskClassifier(
    len(offsides_tasks), n_features, **model_params)
    return model

In [ ]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
optimizer = dc.hyper.HyperparamOpt(model_builder)
best_dnn, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict, train_dataset, valid_dataset, [], metric)

Fitting model 1/6
hyperparameters: {'activation': 'relu', 'optimizer': 'Adam', 'momentum': 0.9, 'penalty': 0.0}
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/deepchem/models/tensorgraph/tensor_graph.py", line 1393, in _enqueue_batch
    sess.run(tg.input_queue.out_tensor, feed_dict=enq)
  File "/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 929, in run
    run_metadata_ptr)
  File "/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1121, in _run
    np_val = np.asarray(subfeed_val, dtype=subfeed_dtype)
  File "/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages

In [60]:
best_dnn

MultitaskClassifier(activation_fns=None, bias_init_consts=None, dropouts=None,
                    layer_sizes=None, n_classes=2, n_features=1024, n_tasks=27,
                    weight_decay_penalty=None, weight_decay_penalty_type=None,
                    weight_init_stddevs=None)

In [61]:
best_hyperparams

('relu', 'Adam', 0.9, 0.0)

In [46]:
all_results

{"('relu', 0.9, 0.0)": 0.673746852484698,
 "('sigmoid', 0.9, 0.0)": 0.677523700567962,
 "('tahn', 0.9, 0.0)": 0.6779358768268917}

In [47]:
model=model_builder(params_dict,params_dict)

In [48]:
model.fit(train_dataset, nb_epoch=100)

154.36382479691386

In [49]:
obj=best_dnn.fit(train_dataset,**params_dict,epochs=100)

In [51]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

In [52]:
train_scores = model.evaluate(train_dataset, [metric], transformers)

computed_metrics: [0.999376477978126, 0.9997829797788218, 1.0, 0.999951157565693, 0.9999160991017668, 0.9995059613991173, 0.9998969201534545, 0.9980603916457189, 0.9999344044697566, 0.9998678200138943, 0.9997858498865583, 0.9999026407536533, 0.9998593158487091, 0.996823499074009, 0.9999209803850739, 0.9999265397273155, 0.9998704998704999, 0.9995396227942367, 0.9998608730494402, 0.9998500336910612, 0.9999192269834472, 0.9995410158848416, 0.9986063565108405, 0.9999275898194367, 0.9999651171329955, 0.9999608664175161, 0.9998507089588513]


In [53]:
valid_scores = model.evaluate(valid_dataset, [metric], transformers)

computed_metrics: [0.6904457973291438, 0.5733415508740476, nan, 0.613184584178499, 0.6409558378705384, 0.6237373737373737, 0.8082706766917294, 0.7155797101449275, 0.5750784401613627, 0.6958137715179968, 0.6271739130434784, 0.5875739644970415, 0.7090579710144927, 0.6139122315592904, 0.6487577639751553, 0.6811244979919678, 0.6596692111959288, 0.5900590551181103, 0.6015070921985816, 0.6687883074021688, 0.6182038834951455, 0.6120430107526882, 0.6736842105263159, 0.676957223567393, 0.6353225806451612, 0.7947658402203857, 0.5948581560283688]


/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Only one class present in y_true. ROC AUC score is not defined in that case.
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


In [54]:
test_scores = model.evaluate(test_dataset, [metric], transformers)

computed_metrics: [0.67573385518591, 0.6939263510282161, 0.3279761904761904, 0.596551724137931, 0.7220625798212005, 0.5354109274563821, 0.6129476584022038, 0.5577673692427791, 0.6051423324150597, 0.5634050880626224, 0.6781553398058253, 0.6484444444444445, 0.5826612903225807, 0.5734295415959254, 0.6637790697674419, 0.6391397849462366, 0.765339966832504, 0.5323004201680672, 0.7463592233009708, 0.632233381157341, 0.6519742883379247, 0.611111111111111, 0.4527671755725191, 0.598995983935743, 0.6976226660877117, 0.596483942414175, 0.6470828233374133]


In [56]:
print(train_scores)
print(valid_scores)
print(test_scores)

{'mean-roc_auc_score': 0.9996075166257347}
{'mean-roc_auc_score': 0.6511487175283575}
{'mean-roc_auc_score': 0.6151409084950529}
